# **PROYECTO MLOPS - STEAM GAMES**

## 1. Importación de Librerías

In [3]:
import pandas as pd # Cargamos la libreria de "pandas" para la manipulación y el análisis de datos
import numpy as np # Cargamos la librería de "numpy" para realizar cálculos lógicos y matemáticos sobre cuadros y matrices en el caso que lo necesitemos
import scipy as sp # Cargamos la librería "scipy"  que proporciona funcionalidades adicionales para tareas científicas y técnicas. Contiene módulos para optimización, álgebra lineal, integración, interpolación, estadísticas, procesamiento de señales y más.
import operator #Cargamos la librería "operator" que proporciona funciones que implementan operaciones similares a los operadores en el lenguaje
from sklearn.metrics.pairwise import cosine_similarity #Cargamos el modelo "cosine_similarity" que se utiliza para calcular la similitud coseno entre conjuntos de datos
from sklearn.feature_extraction.text import TfidfVectorizer #Cargamos el modelo "TfidVectorizer" se utiliza para convertir una colección de documentos de texto en una matriz TF-IDF (Term Frequency-Inverse Document Frequency)
import pyarrow.parquet as pq
import pyarrow as pa
import warnings
warnings.filterwarnings("ignore")


## 2. MLOPS - Desarrollo de Proyecto

### 2.1. Carga de DataSet

In [4]:
#Establecemos las rutas de los archivos
ruta_steam_games_parquet = r'Dataset_Clean\steam_games_clean.parquet'
ruta_user_reviews_parquet = r'Dataset_Clean\australian_user_reviews_sentanaly_clean.parquet'
ruta_user_items_parquet = r'Dataset_Clean\australian_user_items_clean.parquet'

#Cargamos los archivos limpios luego de hacer el ETL
df_steam_games = pd.read_parquet(ruta_steam_games_parquet) #Cargamos el archivo steam_games_clean
df_user_reviews = pd.read_parquet(ruta_user_reviews_parquet) #Cargamos el archivo australian_user_reviews
df_user_items = pd.read_parquet(ruta_user_items_parquet) #Cargamos el archivo australian_user_items

print(f'Se han leido exitosamente los siguientes archivos parquet')

Se han leido exitosamente los siguientes archivos parquet


### 2.2. Analisís Descriptivo de los DataSet

#### 2.2.1. Definición de las Funciones Descriptivas de los DataSet

In [5]:
#Definimos algunas funciones para que nos facilita la descripcion de las principales caracteristicas del DataFrame
def caracteristicas_df(df):
    """
    Describe de forma general la base de datos .

    Esta función simplemente muestra el tamaño, información general y
    la cantidad de datos nulos.

    Parametros
    ----------
    df (pandas.DataFrame): El DataFrame que se va a analizar.

    Returns:
    ----------
        - 'df.shape': Numero de filas y columnas
        - 'df.info': Muestra información general del DataFrame

    """
    print('*'*10 + '|'*10 + 'FORMA DE BASE DE DATOS' + '|'*10 + '*'*10, end = '\n'*2)
    print(f'Tiene {df.shape[0]} filas y {df.shape[1]} columnas o variables')
    print(end = '\n'*2)

    print('*'*10 + '|'*10 + 'INFORMACION GENERAL DE LA BASE DE DATOS' + '|'*10 + '*'*10, end = '\n'*2)
    print(df.info(), end = '\n'*2)

def valores_nulos_df(df):
    """
    Revisa presencia de valores nulos en un DataFrame.
    Esta función toma un DataFrame como entrada y devuelve un resumen que incluye información sobre
    el porcentaje de valores no nulos y nulos, así como la ncantidad de valores nulos por columna.

    Parametros:
    ----------
    df (pandas.DataFrame): El DataFrame que se va a analizar.

    Returns:
    ----------
        pandas.DataFrame: Un DataFrame que contiene el resumen de cada columna, incluyendo:
        - 'nombre': Nombre de cada columna.
        - 'no_nulos_%': Porcentaje de valores no nulos en cada columna.
        - 'nulos_%': Porcentaje de valores nulos en cada columna.
        - 'nulos': Cantidad de valores nulos en cada columna.

    """
    mi_df = {"nombre": [], "tipo_datos": [], "nulos_%": [], "nulos": []}

    for columna in df.columns:
        porcentaje_no_nulos = (df[columna].count() / len(df)) * 100
        mi_df["nombre"].append(columna)
        mi_df["tipo_datos"].append(df[columna].apply(type).unique())
        mi_df["nulos_%"].append(round(100-porcentaje_no_nulos, 2))
        mi_df["nulos"].append(df[columna].isnull().sum())

    df_nulos = pd.DataFrame(mi_df)

    return df_nulos

def cant_porcentaje(df, columna):

    """
    Cuanta la cantidad de True/False luego calcula el porcentaje.

    Parameters:
    ----------
    - df (DataFrame): El DataFrame que contiene los datos.
    - columna (str): El nombre de la columna en el DataFrame para la cual se desea generar el resumen.

    Returns:
    ----------
    DataFrame: Un DataFrame que resume la cantidad y el porcentaje de True/False en la columna especificada.

    """
    # Cuenta la cantidad de True/False luego calcula el porcentaje
    counts = df[columna].value_counts()
    percentages = round(100 * counts / len(df),2)
    # Crea un dataframe con el resumen
    df_results = pd.DataFrame({
        "Cantidad": counts,
        "Porcentaje_%": percentages
    })
    return df_results

def verifica_duplicados(df, columna):
    '''
    Verifica y muestra filas duplicadas en un DataFrame basado en una columna específica.

    Esta función toma como entrada un DataFrame y el nombre de una columna específica.
    Luego, identifica las filas duplicadas basadas en el contenido de la columna especificada,
    las filtra y las ordena para una comparación más sencilla.

    Parameters:
    ----------
        df (pandas.DataFrame): El DataFrame en el que se buscarán filas duplicadas.
        columna (str): El nombre de la columna basada en la cual se verificarán las duplicaciones.

    Returns:
    ----------
        pandas.DataFrame or str: Un DataFrame que contiene las filas duplicadas filtradas y ordenadas,
        listas para su inspección y comparación, o el mensaje "No hay duplicados" si no se encuentran duplicados.
    '''
    # Se filtran las filas duplicadas
    duplicated_rows = df[df.duplicated(subset=columna, keep=False)]
    if duplicated_rows.empty:
        return "No hay duplicados"

    # se ordenan las filas duplicadas para comparar entre sí
    duplicated_rows_sorted = duplicated_rows.sort_values(by=columna)
    return duplicated_rows_sorted

#### 3.2.2. Descripción de los DataSet

In [6]:
#Llamamos a la función creada para visualizar las caracteristicas generales del DataSet "STEAM-GAMES"
caracteristicas_df(df_steam_games)

**********||||||||||FORMA DE BASE DE DATOS||||||||||**********

Tiene 71549 filas y 7 columnas o variables


**********||||||||||INFORMACION GENERAL DE LA BASE DE DATOS||||||||||**********

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71549 entries, 0 to 71548
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        71549 non-null  object 
 1   app_name      71549 non-null  object 
 2   release_date  71549 non-null  object 
 3   price         71549 non-null  float64
 4   early_access  71549 non-null  bool   
 5   id            71549 non-null  int64  
 6   developer     71549 non-null  object 
dtypes: bool(1), float64(1), int64(1), object(4)
memory usage: 3.3+ MB
None



In [7]:
#Llamamos a la función creada para visualizar las caracteristicas generales del DataSet "STEAM-GAMES"
valores_nulos_df(df_steam_games)

,nombre,tipo_datos,nulos_%,nulos
0,genres,[<class 'str'>],0.0,0
1,app_name,[<class 'str'>],0.0,0
2,release_date,[<class 'str'>],0.0,0
3,price,[<class 'float'>],0.0,0
4,early_access,[<class 'bool'>],0.0,0
5,id,[<class 'int'>],0.0,0
6,developer,[<class 'str'>],0.0,0


In [8]:
#Llamamos a la función creada para visualizar las caracteristicas generales del DataSet "USER-REVIEWS"
caracteristicas_df(df_user_reviews)

**********||||||||||FORMA DE BASE DE DATOS||||||||||**********

Tiene 53902 filas y 11 columnas o variables


**********||||||||||INFORMACION GENERAL DE LA BASE DE DATOS||||||||||**********

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53902 entries, 0 to 53901
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   user_id               53902 non-null  object
 1   user_url              53902 non-null  object
 2   item_id               53902 non-null  int64 
 3   recommend             53902 non-null  int64 
 4   review                53604 non-null  object
 5   Posted Date           53902 non-null  object
 6   Date last edited      53902 non-null  object
 7   funny review votes    53902 non-null  int64 
 8   Helpful review votes  53902 non-null  int64 
 9   total review votes    53902 non-null  int64 
 10  sentiment_analysis    53902 non-null  int64 
dtypes: int64(6), object(5)
memory usage: 4.5+ MB


In [9]:
#Llamamos a la función creada para visualizar las caracteristicas generales del DataSet "USER_REVIEWS"
valores_nulos_df(df_user_reviews)

,nombre,tipo_datos,nulos_%,nulos
0,user_id,[<class 'str'>],0.00,0
1,user_url,[<class 'str'>],0.00,0
2,item_id,[<class 'int'>],0.00,0
3,recommend,[<class 'int'>],0.00,0
4,review,"[<class 'str'>, <class 'NoneType'>]",0.55,298
5,Posted Date,[<class 'str'>],0.00,0
6,Date last edited,[<class 'str'>],0.00,0
7,funny review votes,[<class 'int'>],0.00,0
8,Helpful review votes,[<class 'int'>],0.00,0
9,total review votes,[<class 'int'>],0.00,0


In [10]:
#Llamamos a la función creada para visualizar las caracteristicas generales del DataSet "USER-ITEMS"
caracteristicas_df(df_user_items)

**********||||||||||FORMA DE BASE DE DATOS||||||||||**********

Tiene 5094105 filas y 8 columnas o variables


**********||||||||||INFORMACION GENERAL DE LA BASE DE DATOS||||||||||**********

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5094105 entries, 0 to 5094104
Data columns (total 8 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   item_id           int64 
 1   item_name         object
 2   playtime_forever  int64 
 3   playtime_2weeks   int64 
 4   user_id           object
 5   items_count       int64 
 6   steam_id          int64 
 7   user_url          object
dtypes: int64(5), object(3)
memory usage: 310.9+ MB
None



In [11]:
#Llamamos a la función creada para visualizar las caracteristicas generales del DataSet "USER-ITEMS"
valores_nulos_df(df_user_items)

,nombre,tipo_datos,nulos_%,nulos
0,item_id,[<class 'int'>],0.0,0
1,item_name,[<class 'str'>],0.0,0
2,playtime_forever,[<class 'int'>],0.0,0
3,playtime_2weeks,[<class 'int'>],0.0,0
4,user_id,[<class 'str'>],0.0,0
5,items_count,[<class 'int'>],0.0,0
6,steam_id,[<class 'int'>],0.0,0
7,user_url,[<class 'str'>],0.0,0


### 2.3. Funciones de OUTPUT


En este espacio, trabajaremos en el desarrollo de las funciones requeridas para nuestro MVP (Producto Mínimo Viable). Posteriormente, haremos las modificaciones necesarias para integrarlas en la implementación final (deploy). Este proceso implica optimizar y ajustar las funciones según las necesidades específicas del despliegue.

#### 2.3.1. Función *developer*



*   def developer( desarrollador : str ): Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora. Ejemplo de retorno:



In [64]:
def developer(desarrollador: str):
    """
    La función `developer` analiza el DataFrame `df_steam_games` para proporcionar información sobre la cantidad
    de juegos y el porcentaje de contenido gratuito por año, según un desarrollador específico.

    Parameters:
    ----------
    - desarrollador (str): Nombre del desarrollador.

    Returns:
    ----------
    Un DataFrame con las columnas 'Año', 'Cantidad de Items' y 'Contenido Free'.

    Pasos:
    ----------
    1. Filtrar las fechas con "dato sin especificar".
    2. Seleccionar juegos gratuitos del desarrollador y agruparlos por año.
    3. Obtener el total de juegos del desarrollador y agruparlos por año.
    4. Calcular el porcentaje de juegos gratuitos respecto al total por año y crear un DataFrame resultante.

    Ejemplo:
    ```
    desarrollador_ejemplo = "NombreDelDesarrollador"
    resultado = developer(desarrollador_ejemplo)
    print(resultado)
    ```
    """

    # Paso 1
    df_filtered = df_steam_games[df_steam_games['release_date'] != "dato sin especificar"]

    # Paso 2
    free_games = df_filtered[(df_filtered['developer'] == desarrollador) & (df_filtered['price'] == 0)].drop_duplicates(subset=['id'])
    free_games = free_games.groupby('release_date')['app_name'].nunique()

    # Paso 3
    all_games = df_filtered[df_filtered.developer == desarrollador].groupby('release_date')['app_name'].nunique()

    # Combinar las series para asegurarse de que todos los años estén presentes en ambas
    combined = pd.merge(all_games, free_games, how='outer', left_index=True, right_index=True, suffixes=('_total', '_free'))

    # Paso 4
    combined['Contenido Free'] = combined['app_name_free'] / combined['app_name_total']

    # Crear el DataFrame resultante
    result = combined.reset_index().rename(columns={'release_date': 'Año', 'app_name_total': 'Cantidad de Items'}).fillna(0)
    
    return result


In [62]:
df_steam_games['developer'].unique()

array(['Kotoshiro', 'Secret Level SRL', 'Poolians.com', ...,
       'Oscar Ortigueira López,OrtiGames/OrtiSoft', 'INGAME',
       'Bidoniera Games'], dtype=object)

In [65]:
#Verificamos si la función tiene un correcto funcionamiento
desarrollador = 'Secret Level SRL'
developer(desarrollador)


,Año,Cantidad de Items,app_name_free,Contenido Free
0,2018,1,1,1.0


#### 2.3.2. Función *userdata*

*   def userdata( User_id : str ): Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.

In [15]:
def userdata(User_id: str):
    """
    La función recibirá un id de usuario y aplicándolo devolverá la cantidad de
    dinero gastado junto al porcentaje de recomendaciones.

    Parameters:
    ----------
    - User_id (str): El identificador del usuario.

    Returns:
    ----------
    Un diccionario con la información del usuario incluyendo el dinero gastado,
    el porcentaje de recomendaciones y la cantidad de items.

    Steps:
    ----------
    1. Con el User_id, se buscan los juegos del usuario en df_user_items y se guardan en 'games'.
    2. Se calcula la cantidad de dinero gastado basado en la columna 'price' de los juegos.
    3. Se cuenta la cantidad total de juegos del usuario.
    4. Se suman las recomendaciones totales del usuario.
    5. Se calcula el porcentaje de recomendaciones.
    6. Se crea el diccionario.

    """
    global df_steam_items  # Accede al DataFrame global

    # Paso 1
    games = df_user_items[df_user_items['user_id'] == User_id]['item_id']
    games = games.tolist()
    spend_money = 0.0

    # Paso 2
    for game in games:
        price_games = df_steam_games.loc[df_steam_games['id'] == game, 'price']
        if not price_games.empty:
            price = price_games.values[0]
            spend_money += float(price)

    # Paso 3
    cant_games = df_user_items[df_user_items['user_id'] == User_id].shape[0]

    # Paso 4
    recomendations = df_user_reviews['recommend'][df_user_reviews['user_id']==User_id].sum()

    # Paso 5
    ratio = recomendations / cant_games

    # Paso 6
    result = {
        "Usuario": User_id,
        "Dinero gastado": f"{spend_money} USD",
        "% de recomendación": f"{round(ratio * 100, 2)}%",
        "Cantidad de items": cant_games
    }

    return result

In [16]:
df_user_items['user_id'].unique()

array(['76561197970982479', 'js41637', 'evcentric', ...,
       '76561198323066619', '76561198326700687', '76561198329548331'],
      dtype=object)

In [17]:
#Verificamos si la función tiene un correcto funcionamiento
User_id = 'js41637'
userdata(User_id)

{'Usuario': 'js41637',
 'Dinero gastado': '8489.139999999881 USD',
 '% de recomendación': '0.34%',
 'Cantidad de items': 888}

#### 2.3.3. Función *UserForGenre*

*   def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.

In [18]:
def UserForGenre(genre: str):
    """
    La función devuelve el usuario que acumula más horas jugadas para el género dado
    y una lista de la acumulación de horas jugadas por año de lanzamiento.

    Parameters:
    ----------
    - genre (str): El género de interés para realizar el filtro.

    Returns:
    ----------
    Un diccionario con la información del usuario con más horas jugadas para el género y la acumulación
    de horas jugadas por año de lanzamiento.
    Ejemplo de retorno: {"Usuario con más horas jugadas para Género X": "us213ndjss09sdf", "Horas jugadas": [{"Año": 2013, "Horas": 203}, {"Año": 2012, "Horas": 100}, {"Año": 2011, "Horas": 23}]}

    Steps:
    ----------
    1. Combinamos los DataFrames df_user_items y df_steam_games mediante el campo 'item_id' y 'id'.
    2. Filtramos los datos combinados para el género específico.
    3. Eliminamos los datos sin especificar en el campo 'release_year'.
    4. Encuentramos el usuario con más horas jugadas para el género.
    5. Filtramos los datos del usuario con más horas jugadas para el género.
    6. Agrupamos las horas jugadas por año de lanzamiento.
    7. Creamos la lista de acumulación de horas jugadas por año en el formato especificado.

    """

    # Paso 1
    data_by_genre = df_user_items.merge(df_steam_games, how='inner', left_on='item_id', right_on='id')

    # Paso 2
    data_by_genre = data_by_genre[data_by_genre['genres'] == genre]

    if data_by_genre.empty:
        return {"Usuario con más horas jugadas para " + genre: None, "Horas jugadas": []}

    # Paso 3
    data_by_genre = data_by_genre[data_by_genre['release_date'] != 'dato sin especificar']

    # Paso 4
    top_user = data_by_genre.groupby(['user_id'])['playtime_forever'].sum().idxmax()

    # Paso 5
    user_data = data_by_genre[data_by_genre['user_id'] == top_user]

    # Paso 6
    hours_by_year = user_data.groupby(['release_date'])['playtime_forever'].sum().reset_index()

    # Paso 7
    hours_list = [{"Año": int(year), "Horas": int(hours)} for year, hours in zip(hours_by_year['release_date'], hours_by_year['playtime_forever'])]

    return {"Usuario con más horas jugadas para " + genre: top_user, "Horas jugadas": hours_list}


In [19]:
df_steam_games['genres'].unique()

array(['Action', 'Casual', 'Indie', 'Simulation', 'Strategy',
       'Free to Play', 'RPG', 'Sports', 'Adventure', 'Racing',
       'Early Access', 'Massively Multiplayer',
       'Animation &amp; Modeling', 'Video Production', 'Utilities',
       'Web Publishing', 'Education', 'Software Training',
       'Design &amp; Illustration', 'Audio Production', 'Photo Editing',
       'Accounting'], dtype=object)

In [20]:
#Verificamos si la función tiene un correcto funcionamiento
genre = 'Action'
UserForGenre(genre)

{'Usuario con más horas jugadas para Action': 'Sp3ctre',
 'Horas jugadas': [{'Año': 1993, 'Horas': 0},
  {'Año': 1995, 'Horas': 217},
  {'Año': 1996, 'Horas': 0},
  {'Año': 1998, 'Horas': 0},
  {'Año': 1999, 'Horas': 44},
  {'Año': 2000, 'Horas': 70644},
  {'Año': 2001, 'Horas': 13},
  {'Año': 2002, 'Horas': 238},
  {'Año': 2003, 'Horas': 7673},
  {'Año': 2004, 'Horas': 127411},
  {'Año': 2005, 'Horas': 21339},
  {'Año': 2006, 'Horas': 896},
  {'Año': 2007, 'Horas': 112784},
  {'Año': 2008, 'Horas': 224},
  {'Año': 2009, 'Horas': 108326},
  {'Año': 2010, 'Horas': 78083},
  {'Año': 2011, 'Horas': 154896},
  {'Año': 2012, 'Horas': 378296},
  {'Año': 2013, 'Horas': 120461},
  {'Año': 2014, 'Horas': 130691},
  {'Año': 2015, 'Horas': 312511},
  {'Año': 2016, 'Horas': 29576},
  {'Año': 2017, 'Horas': 43327}]}

#### 2.3.4. Función *best_developer_year*

*   def best_developer_year( año : int ): Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos)

In [68]:
def best_developer_year(anio: int):
    """
    La función retorna el top 3 de desarrolladores con juegos MÁS recomendados por usuarios
    para el año dado, considerando solo revisiones recomendadas y con comentarios positivos.

    Parameters:
    ----------
    - año (int): Año para el cual se desea obtener el top 3 de desarrolladores.

    Returns:
    ----------
    Una lista de diccionarios en el formato [{"Puesto {}: {}".format(i + 1, desarrollador)}]
    donde i es la posición en el top y desarrollador es el nombre del desarrollador.

    Steps:
    ----------
    1. Filtramos los juegos del año específico digitado, excluyendo "dato sin especificar".
    2. Convertimos la columna 'release_date' a tipo datetime.
    3. Filtramos los juegos por el año específico.
    4. Fusionamos los dataframes para obtener los juegos y sus recomendaciones correspondientes.
    5. Filtramos revisiones recomendadas y comentarios positivos.
    6. Contamos la cantidad de revisiones positivas para cada desarrollador.
    7. Obtenemos los 3 principales desarrolladores o el top 3 de desarrolladores.
    8. Creamos la lista de retorno en el formato especificado.

    """

    # Paso 1
    juegos_del_año = df_steam_games[(df_steam_games['release_date'] != 'dato sin especificar') & (df_steam_games['release_date'].str.isnumeric())]

    # Paso 2
    juegos_del_año['release_date'] = pd.to_numeric(juegos_del_año['release_date'], errors='coerce')

    # Paso 3
    juegos_del_año = juegos_del_año[juegos_del_año['release_date'] == int(anio)]

    # Paso 4
    df = pd.merge(juegos_del_año, df_user_reviews, left_on='id', right_on='item_id')

    # Paso 5
    df_recomendado_positivo = df[(df['recommend'] == 1) & (df['sentiment_analysis'] == 2)]

    # Paso 6
    desarrolladores_con_revisiones = df_recomendado_positivo['developer'].value_counts()

    # Paso 7
    top_3_desarrolladores = desarrolladores_con_revisiones.head(3).index.tolist()

    # Paso 8
    resultado = [{"Puesto {}: {}".format(i + 1, desarrollador)} for i, desarrollador in enumerate(top_3_desarrolladores)]

    return resultado

In [69]:
df_steam_games['release_date'].unique()

array(['2018', '2017', 'dato sin especificar', '1997', '1998', '2016',
       '2006', '2005', '2003', '2007', '2002', '2000', '1995', '1996',
       '1994', '2001', '1993', '2004', '1999', '2008', '2009', '1992',
       '1989', '2010', '2011', '2013', '2012', '2014', '1983', '1984',
       '2015', '1990', '1988', '1991', '1987', '1986', '2021', '2019',
       '1985'], dtype=object)

In [71]:
#Verificamos si la función tiene un correcto funcionamiento
año = 2013
best_developer_year(año)

[{'Puesto 1: Facepunch Studios'},
 {'Puesto 2: Bohemia Interactive'},
 {'Puesto 3: Digital Extremes'}]

#### 2.3.5. Función *developer_reviews_analysis*

*   def developer_reviews_analysis( desarrolladora : str ): Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.


In [24]:
def developer_reviews_analysis(desarrolladora: str):
    """
    Esta función retorna un diccionario con la cantidad total de registros de reseñas de usuarios
    categorizados con un análisis de sentimiento como positivo o negativo para el desarrollador dado.

    Parameters:
    ----------
    - desarrolladora (str): Nombre del desarrollador para el cual se desea realizar el análisis.

    Returns:
    ----------
    Un diccionario en el formato {desarrolladora: {'Positive': cantidad_positivas, 'Negative': cantidad_negativas}}
    donde 'cantidad_positivas' y 'cantidad_negativas' son la cantidad total de reseñas positivas y negativas, respectivamente.

    Steps:
    ----------
    1. Filtramos las reseñas del desarrollador especificado.
    2. Fusionamos los dataframes para obtener las reseñas correspondientes al desarrollador.
    3. Contamos las reseñas positivas y negativas.
    4. Creamos el diccionario de retorno.
    """

    # Paso 1 y 2
    developer_reviews = df_steam_games[df_steam_games['developer'] == desarrolladora].merge(
        df_user_reviews, left_on='id', right_on='item_id', how='inner'
    )

    # Paso 3
    positive_reviews = developer_reviews[developer_reviews['sentiment_analysis'] == 2].shape[0]
    negative_reviews = developer_reviews[developer_reviews['sentiment_analysis'] == 0].shape[0]

    # Paso 4
    result_dict = {desarrolladora: {'Positive': positive_reviews, 'Negative': negative_reviews}}

    return result_dict

In [25]:
df_steam_games['developer'].unique()

array(['Kotoshiro', 'Secret Level SRL', 'Poolians.com', ...,
       'Oscar Ortigueira López,OrtiGames/OrtiSoft', 'INGAME',
       'Bidoniera Games'], dtype=object)

In [26]:
#Verificamos si la función tiene un correcto funcionamiento
desarrolladora = 'Valve'
developer_reviews_analysis(desarrolladora)

{'Valve': {'Positive': 8664, 'Negative': 1218}}

## 2.4. Modelo de Recomendación

Nos han solicitado un modelo de Machine Learning para armar un sistema de recomendación. Para ello, te ofrecen dos propuestas de trabajo: En la primera, el modelo deberá tener una relación ítem-ítem, esto es se toma un item, en base a que tan similar esa ese ítem al resto, se recomiendan similares. Aquí el input es un juego y el output es una lista de juegos recomendados, para ello recomendamos aplicar la similitud del coseno. La otra propuesta para el sistema de recomendación debe aplicar el filtro user-item, esto es tomar un usuario, se encuentran usuarios similares y se recomiendan ítems que a esos usuarios similares les gustaron.

#### 2.4.1. Preparación de Modelo

In [27]:
#Función de Transformación
def calcula_rating(row):
    '''
    Calcula una calificación basada en el análisis de sentimientos y la recomendación de reseñas de juegos realizadas por los usuarios.

    Parameters:
    ----------
        row (dict): Un diccionario que contiene las siguientes claves:
            - "sentiment_analysis" (int): La puntuación del análisis de sentimientos (0, 1 o 2).
            - "reviews_recommend" (bool): Indica si las reseñas recomiendan.

    Returns:
    ----------
        int o None: La calificación calculada como un número entero entre 1 y 5, o None si las entradas son inválidas.

      -1 si el análisis de sentimiento es negativo ya sea que este recomendado o no (True o False).
      -2 si el análisis de sentimiento es neutral y no es recomendado (False).
      -3 si el análisis de sentimiento es neutral pero es recomendado (True).
      -4 si el análisis de sentimiento es positivo y no es recomendado (False).
      -5 si el análisis de sentimiento es positivo y es recomendado (True).

    '''
    sentiment = row["sentiment_analysis"]
    recommend = row["recommend"]

    if sentiment == 0:
        return 1 if not recommend else 1
    elif sentiment == 1:
        return 2 if not recommend else 3
    elif sentiment == 2:
        return 4 if not recommend else 5
    else:
        return None

In [28]:
#Creamos una nueva columna "rating"
df_user_reviews['rating'] = df_user_reviews.apply(calcula_rating, axis=1)

In [29]:
#Creamos una mascara solo con las columnas "user_id","reviews_items_id" y "rating"
df1 = df_user_reviews[['user_id', 'item_id', 'rating']]

In [30]:
#Creamos una mascara solo con las columnas "item_id" y "item_name"
df2 = df_user_items[['item_id', 'item_name']]
#Eliminamos los duplicados de juegos
df2 = df2.drop_duplicates()

In [31]:
#Luego hacemos un merge para unir ambas DataFrame
df = df1.merge(df2, left_on="item_id", right_on="item_id", how='left')
df

,user_id,item_id,rating,item_name
0,76561197970982479,1250,5,Killing Floor
1,76561197970982479,22200,5,Zeno Clash
2,76561197970982479,43110,5,Metro 2033
3,js41637,251610,5,Barbie‚Ñ¢ Dreamhouse Party‚Ñ¢
4,js41637,227300,5,Euro Truck Simulator 2
...,...,...,...,...
53897,76561198312638244,130,5,Half-Life: Blue Shift
53898,76561198312638244,362890,5,Black Mesa
53899,LydiaMorley,273110,1,Counter-Strike Nexon: Zombies
53900,LydiaMorley,730,5,Counter-Strike: Global Offensive


In [32]:
#Verificamos si existe algún valor nulo
valores_nulos_df(df)

,nombre,tipo_datos,nulos_%,nulos
0,user_id,[<class 'str'>],0.0,0
1,item_id,[<class 'int'>],0.0,0
2,rating,[<class 'int'>],0.0,0
3,item_name,"[<class 'str'>, <class 'float'>]",11.2,6037


In [33]:
# Se borran los nulos
df = df.dropna(subset=['item_name'])

# Se verifican los tipo de dato y nulos
valores_nulos_df(df)

,nombre,tipo_datos,nulos_%,nulos
0,user_id,[<class 'str'>],0.0,0
1,item_id,[<class 'int'>],0.0,0
2,rating,[<class 'int'>],0.0,0
3,item_name,[<class 'str'>],0.0,0


In [34]:
#Creamos un ultimo dataframe para nuestro modelo
#Se procedio a elegir "item_name" en vez de "item_id" para un mejor entendimiento a la hora de hacer la funcion
df = df[['user_id', 'item_name', 'rating']]
df

,user_id,item_name,rating
0,76561197970982479,Killing Floor,5
1,76561197970982479,Zeno Clash,5
2,76561197970982479,Metro 2033,5
3,js41637,Barbie‚Ñ¢ Dreamhouse Party‚Ñ¢,5
4,js41637,Euro Truck Simulator 2,5
...,...,...,...
53896,76561198312638244,Far Cry¬Æ 3 Blood Dragon,5
53897,76561198312638244,Half-Life: Blue Shift,5
53898,76561198312638244,Black Mesa,5
53899,LydiaMorley,Counter-Strike Nexon: Zombies,1


In [35]:
#Por ultimo exportamos el nuevo DataFrame que nos servira para nuestro modelo de recomendación
#Se guarda el dataframe transformado como df_recomendaciones
archivo_limpio = r'Dataset_Clean\df_recomendaciones.csv'
df.to_csv(archivo_limpio, index=False, encoding='utf-8')
print(f'Se guardó el archivo {archivo_limpio}')

Se guardó el archivo Dataset_Clean\df_recomendaciones.csv


In [36]:
#Se guarda el dataframe transformado como australian_user_reviews_clean
archivo_limpio_csv = r'Dataset_Clean\df_recomendaciones.csv'
archivo_limpio_parquet = r'Dataset_Clean\df_recomendaciones.parquet'
# Leemos el archivo CSV en un DataFrame
df2 = pd.read_csv(archivo_limpio_csv)
df2.to_parquet(archivo_limpio_parquet, engine='pyarrow')
print(f'Se guardó el archivo {archivo_limpio_parquet}')

Se guardó el archivo Dataset_Clean\df_recomendaciones.parquet


#### 2.4.2. Desarrollo del Modelo

In [37]:
#Establecemos las rutas de los archivos
ruta_df_recomendaciones_parquet = r'Dataset_Clean\df_recomendaciones.parquet'

#Cargamos los archivos limpios luego de hacer el ETL
df_modelo_recomendacion = pd.read_parquet(ruta_df_recomendaciones_parquet) #Cargamos el archivo steam_games_clean

print(f'Se han leido exitosamente los siguientes archivos parquet')

Se han leido exitosamente los siguientes archivos parquet


In [38]:
print(df_modelo_recomendacion.columns)

Index(['user_id', 'item_name', 'rating'], dtype='object')


In [39]:
# Creamos una tabla pivote con usuarios en filas, juegos en columnas y valores de calificación
tabla_pivote = df_modelo_recomendacion.pivot_table(index=['user_id'], columns=['item_name'], values='rating')

# Normalizamos el dataframe 'tabla_pivote' por filas para tener valores entre 0 y 1
piv_norm = tabla_pivote.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)), axis=1)

# Rellenamos los valores NaN con 0 después de la normalización y transponer la tabla
piv_norm.fillna(0, inplace=True)
piv_norm = piv_norm.T

# Eliminamos las columnas que contienen solo ceros o no tienen calificación
piv_norm = piv_norm.loc[:, (piv_norm != 0).any(axis=0)]

# Creamos una matriz dispersa comprimida (CSR) para reducir el uso de memoria
piv_sparse = sp.sparse.csr_matrix(piv_norm.values)

# Calculamos similitud coseno entre juegos (item similarity) y entre usuarios (user similarity)
item_similarity = cosine_similarity(piv_sparse)
user_similarity = cosine_similarity(piv_sparse.T)

# Creamos dataframes de similitud coseno entre juegos y entre usuarios
juego_df = pd.DataFrame(item_similarity, index=piv_norm.index, columns=piv_norm.index)
usuario_df = pd.DataFrame(user_similarity, index=piv_norm.columns, columns=piv_norm.columns)

In [40]:
#Se guarda los DataFrames finales para luego usarlas en el API en csv
archivo_limpio_juegos = r'Dataset_Clean\juego_df_clean.csv'
archivo_limpio_usuario = r'Dataset_Clean\usuario_df_clean.csv'
juego_df.to_csv(archivo_limpio_juegos, index=False, encoding='utf-8')
usuario_df.to_csv(archivo_limpio_usuario, index=False, encoding='utf-8')
print(f'Se guardó el archivo {archivo_limpio_juegos} y {archivo_limpio_usuario}')

Se guardó el archivo Dataset_Clean\juego_df_clean.csv y Dataset_Clean\usuario_df_clean.csv


In [41]:
#Se guarda los DataFrames finales para luego usarlas en el API en parquet
juego_df_parquet = r'Dataset_Clean\juego_df_clean.parquet'
usuario_df_parquet = r'Dataset_Clean\usuario_df_clean.parquet'

# Exportamos en parquet
pq.write_table(pa.Table.from_pandas(juego_df), juego_df_parquet)
pq.write_table(pa.Table.from_pandas(usuario_df), usuario_df_parquet)

print(f'Se guardó el archivo {juego_df_parquet} y {usuario_df_parquet}')

Se guardó el archivo Dataset_Clean\juego_df_clean.parquet y Dataset_Clean\usuario_df_clean.parquet


#### 2.4.3. Funciones de Recomendación

In [73]:
def recomendacion_juego(juego):
    '''
    Esta función muestra una lista de juegos similares a un juego dado.

    Parameters:
    ----------
        juego (str): El nombre del juego para el cual se desean encontrar juegos similares.

    Returns:
    ----------
        None: Esta función imprime una lista de juegos 5 similares al dado.

    Pasos:
    ----------
    1. Verificamos si el juego está en el DataFrame de similitud
    2. Obtenemos la lista de juegos similares y mostrarlos
    3. Imprimimos la lista de juegos similares

    '''

    # Paso 1
    if juego not in juego_df.index:
        print(f'No se encontraron juegos similares para {juego}.')
        return

    # Paso 2
    similar_juegos = juego_df.sort_values(by=juego, ascending=False).index[1:6]  # Mostrar siempre los primeros 5

    # Paso 3
    juegos_similares = [item for item in similar_juegos]

    return juegos_similares 

In [74]:
df_user_items['item_name'].unique()

array(['Counter-Strike', 'Team Fortress Classic', 'Day of Defeat', ...,
       'ChaosTower', 'Aveyond 4: Shadow Of The Mist', 'Arachnophobia'],
      dtype=object)

In [77]:
#Verificamos si la función tiene un correcto funcionamiento
juego = 'Day of Defeat'
recomendacion_juego(juego)

['Halo: Spartan Assault',
 'Thinking with Time Machine',
 'PAYDAY: The Heist',
 'Arma: Cold War Assault',
 'Revenge of the Titans']

In [45]:
def recomendacion_usuario(usuario):
    '''
    Esta función genera una lista de los juegos más recomendados para un usuario, basándose en las calificaciones de usuarios similares.

    Parameters:
    ----------
        usuario (str): El nombre o identificador del usuario para el cual se desean generar recomendaciones.

    Returns:
    ----------
        list: Una lista de los juegos más recomendados para el usuario basado en la calificación de usuarios similares.

    Pasos:
    ----------
    1. Verificamos si el usuario está presente en las columnas de piv_norm.
    2. Obtenemos los usuarios más similares al usuario dado.
    3. Para cada usuario similar, encuentra el juego mejor calificado y lo agrega a la lista 'mejores_juegos'.
    4. Contamos cuántas veces se recomienda cada juego.
    5. Ordenamos los juegos por la frecuencia de recomendación en orden descendente.
    6. Devolvemos los 5 juegos más recomendados.


    '''
    # Paso 1
    if usuario not in piv_norm.columns:
        print(f'No hay datos disponibles para el usuario {usuario}.')
        return

    # Paso 2
    sim_users = usuario_df.sort_values(by=usuario, ascending=False).index[1:11]

    mejores_juegos = []  # Lista para almacenar los juegos mejor calificados por usuarios similares
    mas_comunes = {}  # Diccionario para contar cuántas veces se recomienda cada juego

    # Paso 3
    for i in sim_users:
        max_score = piv_norm.loc[:, i].max()
        mejores_juegos.append(piv_norm[piv_norm.loc[:, i] == max_score].index.tolist())

    # Paso 4
    for i in range(len(mejores_juegos)):
        for j in mejores_juegos[i]:
            if j in mas_comunes:
                mas_comunes[j] += 1
            else:
                mas_comunes[j] = 1

    # Paso 5
    sorted_list = sorted(mas_comunes.items(), key=operator.itemgetter(1), reverse=True)

    # Paso 6
    return sorted_list[:5]

In [46]:
df_user_items['user_id'].unique()

array(['76561197970982479', 'js41637', 'evcentric', ...,
       '76561198323066619', '76561198326700687', '76561198329548331'],
      dtype=object)

In [47]:
#Verificamos si la función tiene un correcto funcionamiento
id_user = 'evcentric'
recomendacion_usuario(id_user)

[('Risk of Rain', 8),
 ('Dr. Langeskov, The Tiger, and The Terribly Cursed Emerald: A Whirlwind Heist',
  1),
 ('The Slaughtering Grounds', 1),
 ('Transistor', 1)]